# Chapter 4 Further Research Q1

### Task: Create your own implementation of `Learner` from scratch, based on the training loop shown in this chapter.

Some basic questions we need to clarify before we start:
1. What is a `Learner`?
- A `Learner` takes in a `DataLoaders` object, a model, an optimization function, a loss function, and optionally any metrics to print
2. What methods should it be able to support?
- A constructor and a `fit` method

Based on this, we will define our `Learner` class as follows:

In [1]:
from fastai.vision.all import *

In [2]:
class Learner:
    def __init__(self, dls, model, opt_func, loss_func, metric, lr):
        self.dls = dls
        self.model = model
        self.optimizer = opt_func(model.parameters(), lr=lr)
        self.loss_func = loss_func
        self.metric = metric
    
    
    def _train_epoch(self):
        """Performs one training cycle through the training dataset."""
        for xb, yb in self.dls.train:
            preds = self.model(xb).sigmoid()
            loss = self.loss_func(preds, yb)
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()


    def _validate_epoch(self):
        metrics = [self.metric(self.model(xb), yb) for xb, yb in self.dls.valid]
        return round(torch.stack(metrics).mean().item(), 4)
    
        
    def fit(self, epochs):
        print("Metrics for each epoch:")
        for _ in range(epochs):
            self._train_epoch()
            metric = self._validate_epoch()
            print(metric)

We will be testing the `Learner` with the sample MNIST dataset, which only contains data for the digits 3 and 7 (this is the dataset used in the textbook). Each image has dimension 28 * 28.

In [3]:
path = Path('/Users/kyungjaelee/.fastai/data/mnist_sample')
Path.BASE_PATH = path
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

In [4]:
# define training set
train_threes = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'3').ls()]).float()/255
train_sevens = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'7').ls()]).float()/255
train_x = torch.cat([train_threes, train_sevens]).view(-1, 28 * 28)
train_y = tensor([1] * len(train_threes) + [0] * len(train_sevens)).unsqueeze(1)

train_dset = list(zip(train_x, train_y))
train_dl = DataLoader(train_dset, batch_size=256)

In [5]:
# define validation set
valid_threes = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()]).float()/255
valid_sevens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()]).float()/255
valid_x = torch.cat([valid_threes, valid_sevens]).view(-1, 28 * 28)
valid_y = tensor([1] * len(valid_threes) + [0] * len(valid_sevens)).unsqueeze(1)

valid_dset = list(zip(valid_x, valid_y))
valid_dl = DataLoader(valid_dset, batch_size=256)

In [6]:
# create DataLoaders
dls = DataLoaders(train_dl, valid_dl)

In [7]:
# create model
model = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
)

In [8]:
# define metric to use to evaluate the model
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [9]:
learn = Learner(dls, nn.Linear(28*28, 1), opt_func=SGD, loss_func=F.l1_loss, metric=batch_accuracy, lr=0.001)

In [10]:
learn.fit(30)

Metrics for each epoch:
0.562
0.7138
0.8106
0.8673
0.888
0.9082
0.9209
0.9288
0.9332
0.9431
0.9465
0.9509
0.9538
0.9553
0.9583
0.9578
0.9583
0.9588
0.9593
0.9597
0.9597
0.9597
0.9597
0.9597
0.9583
0.9583
0.9583
0.9593
0.9593
0.9593


We can see from the increasing accuracy that our `Learner` class is functioning as intended.